To establish communication between Visual Odometry and Kalman filter, homogeneous transformation matrix is needed.
Currently, filters that adopt velocity motion model contains position and angle in x and z-axis to estimate the motion of an agent in a system.

To compute homogeneous transformation matrix, translation vector that contains position in x, y, and z coordinates, and rotation matrix, which can be derived from quaternion, are needed.
In this notebook, mathematical equation that explains new kinematic equation, which conforms to the velocity motion model, is derived.

When converting euler angles from quaternion, gimbal lock, which occurs when pitch angle (angle in y-axis) is +90 or -90 degree, needs to be taken into account.

In [38]:
import numpy as np
import sympy
from sympy.abc import alpha, x, y, v, w, R, psi, theta, phi
from sympy import symbols, Matrix

In [39]:
# state vector
px = symbols('p_{x}')
py = symbols('p_{y}')
pz = symbols('p_{z}')

vx = symbols('v_{x}')
vy = symbols('v_{y}')
vz = symbols('v_{z}')

qw = symbols('q_{w}')
qx = symbols('q_{x}')
qy = symbols('q_{y}')
qz = symbols('q_{z}')

# control input

ax = symbols('a_{x}')
ay = symbols('a_{y}')
az = symbols('a_{z}')
wx = symbols('w_{x}')
wy = symbols('w_{y}')
wz = symbols('w_{z}')

g = Matrix([
    [0],[0],[9.81]
])

# delta time
dt = symbols('dt')

In [40]:
fv = sympy.sqrt(vx**2 + vy**2 + vz**2)
fv

sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)

In [41]:
norm_w = symbols('|w|')
R = Matrix([
    [qw**2 + qx**2 - qy**2 - qz**2, 2*(qx*qy - qw*qz), 2*(qw*qy + qx*qz)],
    [2*(qx*qy + qw*qz), qw**2 - qx**2 + qy**2 - qz**2, 2*(qy*qz - qw*qx)],
    [2*(qx*qz - qw*qy), 2*(qw*qx + qy*qz), qw**2 - qx**2 - qy**2 + qz**2]
])
Omega = Matrix([
    [0, wz, -wy, wx],
    [-wz, 0, wx, wy],
    [wy, -wx, 0, wz],
    [-wx, -wy, -wz, 0]
])

## Normal condition

In [42]:
phi = sympy.atan2(2*(qw*qx + qy*qz), qw**2 - qx**2 - qy**2 + qz**2)
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = sympy.atan2(2*(qw*qz + qx*qy), qw**2 + qx**2 - qy**2 - qz**2)

In [43]:
A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = Matrix([
    [qw],
    [qx],
    [qy],
    [qz]
])
(A + B * Omega) * q_t

Matrix([
[ q_{w}*cos(dt*|w|/2) + q_{x}*w_{z}*sin(dt*|w|/2)/|w| - q_{y}*w_{y}*sin(dt*|w|/2)/|w| + q_{z}*w_{x}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{z}*sin(dt*|w|/2)/|w| + q_{x}*cos(dt*|w|/2) + q_{y}*w_{x}*sin(dt*|w|/2)/|w| + q_{z}*w_{y}*sin(dt*|w|/2)/|w|],
[ q_{w}*w_{y}*sin(dt*|w|/2)/|w| - q_{x}*w_{x}*sin(dt*|w|/2)/|w| + q_{y}*cos(dt*|w|/2) + q_{z}*w_{z}*sin(dt*|w|/2)/|w|],
[-q_{w}*w_{x}*sin(dt*|w|/2)/|w| - q_{x}*w_{y}*sin(dt*|w|/2)/|w| - q_{y}*w_{z}*sin(dt*|w|/2)/|w| + q_{z}*cos(dt*|w|/2)]])

In [44]:
fxu = Matrix([
    [px - fv*sympy.sin(psi)/wz + fv*sympy.sin(psi + wz*dt)/wz],
    [py + fv*sympy.cos(psi)/wz - fv*sympy.cos(psi + wz*dt)/wz],
    [pz - fv*sympy.sin(phi)/wx + fv*sympy.sin(phi + wx*dt)/wx],
    [Matrix([[vx],[vy], [vz]]) + (R * Matrix([[ax],[ay], [az]]) - g) * dt],
    [(A + B * Omega) * q_t]
])
fxu

Matrix([
[            p_{x} - (2*q_{w}*q_{z} + 2*q_{x}*q_{y})*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z}],
[p_{y} - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2))],
[            p_{z} - (2*q_{w}*q_{x} + 2*q_{y}*q_{z})*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)/(w_{x}*sqrt((2*q_{w}*q_{x} + 2*q_{y}*q_{z})**2 + (q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2)**2)) + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{x} + atan2(2*q_{w}*q_{x} + 2*q_{y}*q_{z}, q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2))/w_{x}]

In [45]:
state_x = Matrix([px, py, pz, vx, vy, vz, qw, qx, qy, qz])
control_input = Matrix([ax, ay, az, wx, wy, wz])

In [46]:
F = fxu.jacobian(state_x)
F

Matrix([
[1, 0, 0,             -v_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) + v_{x}*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)),             -v_{y}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) + v_{y}*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)),             -v_{z}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) + v_{z}*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/(w_{z}*sqrt(v_{x}**

In [47]:
G = fxu.jacobian(control_input)
G

Matrix([
[                                             0,                                              0,                                              0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                         0,                        0,             dt*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z} + (2*q_{w}*q_{z} + 2*q_{x}*q_{y})*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)/(w_{z}**2*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - sqrt(v_{x}**2 + v_{y}**2 + v_{z}*

In [95]:
G[9, :]

Matrix([[0, 0, 0, -q_{w}*sin(dt*|w|/2)/|w|, -q_{x}*sin(dt*|w|/2)/|w|, -q_{y}*sin(dt*|w|/2)/|w|]])

In [22]:
values = {
    px: 0,
    py: 0,
    pz: 0,
    vx: 0,
    vy: 0,
    vz: 0,
    qw: 1,
    qx: 0,
    qy: 0,
    qz: 0,
    v: 1,
    ax: 0.1,
    ay: 0.1,
    az: -9.81,
    wx: 1,
    wy: 1,
    wz: 1,
    dt: 0.1,
    norm_w: np.sqrt(1+1+1)
}

x_ = np.array(fxu.evalf(subs=values)).astype(float)
x_

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.01      ],
       [ 0.01      ],
       [-1.962     ],
       [ 0.99625234],
       [-0.04993752],
       [ 0.04993752],
       [-0.04993752]])

## when pitch = + pi/2 radian

https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html

In [24]:
phi = 0
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = -2*sympy.atan2(qx, qw)

A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = Matrix([
    [qw],
    [qx],
    [qy],
    [qz]
])
fxu = Matrix([
    [px - fv*sympy.sin(psi)/wz + fv*sympy.sin(psi + wz*dt)/wz],
    [py + fv*sympy.cos(psi)/wz - fv*sympy.cos(psi + wz*dt)/wz],
    [pz - fv*sympy.sin(phi)/wx + fv*sympy.sin(phi + wx*dt)/wx],
    [Matrix([[vx],[vy], [vz]]) + (R * Matrix([[ax],[ay], [az]]) - g) * dt],
    [(A + B * Omega) * q_t]
])
F = fxu.jacobian(state_x)
G = fxu.jacobian(control_input)

In [25]:
fxu

Matrix([
[p_{x} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(2*atan2(q_{x}, q_{w}))/w_{z}],
[p_{y} - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                                                                p_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{x})/w_{x}],
[                 dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + v_{x}],
[                 dt*(a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + v_{y}],
[          dt*(a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + a_{y}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + a_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2) - 9.81) + v_{z}],
[      

In [26]:
F

Matrix([
[1, 0, 0,  v_{x}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) + v_{x}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)),  v_{y}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) + v_{y}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)),  v_{z}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) + v_{z}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)), 2*q_{x}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{x}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{w}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                                

In [27]:
G

Matrix([
[                                             0,                                              0,                                              0,                                                                                                                         0,                        0, dt*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                             0,                                              0,                                              0,                                                                                                                         0,                        0, dt*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} - 2*atan2(q_{x}, q_{

## When pitch = - pi/2 radians

https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html

In [28]:
phi = 0
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = 2*sympy.atan2(qx, qw)
A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = Matrix([
    [qw],
    [qx],
    [qy],
    [qz]
])
fxu = Matrix([
    [px - fv*sympy.sin(psi)/wz + fv*sympy.sin(psi + wz*dt)/wz],
    [py + fv*sympy.cos(psi)/wz - fv*sympy.cos(psi + wz*dt)/wz],
    [pz - fv*sympy.sin(phi)/wx + fv*sympy.sin(phi + wx*dt)/wx],
    [Matrix([[vx],[vy], [vz]]) + (R * Matrix([[ax],[ay], [az]]) - g) * dt],
    [(A + B * Omega) * q_t]
])
F = fxu.jacobian(state_x)
G = fxu.jacobian(control_input)

In [29]:
fxu

Matrix([
[p_{x} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(2*atan2(q_{x}, q_{w}))/w_{z}],
[p_{y} - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                                                                p_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{x})/w_{x}],
[                 dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + v_{x}],
[                 dt*(a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + v_{y}],
[          dt*(a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + a_{y}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + a_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2) - 9.81) + v_{z}],
[      

In [30]:
F

Matrix([
[1, 0, 0,  v_{x}*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) - v_{x}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)),  v_{y}*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) - v_{y}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)),  v_{z}*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)) - v_{z}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)), -2*q_{x}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{x}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), 2*q_{w}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{w}*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                                

In [31]:
G

Matrix([
[                                             0,                                              0,                                              0,                                                                                                                         0,                        0, dt*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} - sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z}**2 + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                             0,                                              0,                                              0,                                                                                                                         0,                        0, dt*sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*sin(dt*w_{z} + 2*atan2(q_{x}, q_{w}))/w_{z} + sqrt(v_{x}**2 + v_{y}**2 + v_{z}**2)*cos(dt*w_{z} + 2*atan2(q_{x}, q_{

In [32]:
np.eye(6).shape

(6, 6)